# Suite2p notebook
This file is read only. Please make a copy in your home directory before modifying this file.

In [2]:
import sys
sys.path.insert(1, './extras')

from parse_xml import parse_xml
from raw2h5 import raw2h5
import logging
import os
import glob
import argparse

## Setup input
Setting the ```run_mode``` variable will control which parts of the pipeline are run.
- all: Run the entire pipeline
- suite2p: Run only the registration/cell extraction part
- deconv: Run the deconvolution algorithm from the MATLAB version of suite2p on the extracted cells

Either insert the path to a file in the ```params_file``` variable, or fill your entries in the db variable below.
Make sure your input/output paths are created before running this.

In [3]:
logging.basicConfig(format="%(asctime)s %(relativeCreated)12d [%(filename)s:%(funcName)15s():%(lineno)s] [%(process)d] %(message)s", 
                    datefmt='%Y-%m-%d %H:%M:%S', level=logging.DEBUG)
run_mode = 'all' # Use 'suite2p' to just run the registration/cell extraction, 'deconv' for just the deconvolution or 'all' for everything
params_file = '' # Place path to params.py here if you want to use a separate file.
if os.path.isfile(params_file):
    with open(params_file) as f:
        code = compile(f.read(), params_file, 'exec')
        exec(code)
else:
    db = []
    
    db.append({
        "mouse_name": "183628",
        "date": "2019-06-11",
        "expts": ['1_001'],
        "raw_dir": "//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/",
        "tif_dir": "E:/Userssamsoon.inayat/S_Drive/Tif_Data/",
        "results_dir": "E:/Users/samsoon.inayat/S_Drive/Processed_Data/",
        "ops": {
            'do_bidiphase': True,
            'bidiphase': 1,
            'bidi_corrected': True,
            'sparse_mode': False,
            'diameter': 15
        }
    })

## Run the actual pipeline

In [4]:
for i in db:
    comb_dir = '_'.join(str(x) for x in i['expts'])
    tif_dir = os.path.join(i['tif_dir'], i['mouse_name'], i['date'], comb_dir)
    results_dir = os.path.join(i['results_dir'], i['mouse_name'], i['date'], comb_dir)
    
    if not os.path.isdir(results_dir):
        logging.info(f'Creating results directory {results_dir}')
        os.makedirs(results_dir)
    
    if 'abf_dir' in i:
        abf_dir = [ os.path.join(i['abf_dir'], i['mouse_name'], i['date'] + "_" + str(x) + ".abf") for x in i['expts'] ]
    else:
        abf_dir = None
    print(f'Running expt {i}')
    print(f'Temp directory: {tif_dir}')
    print(f'Results directory: {results_dir}')
    i['in_dir'] = os.path.join(i['raw_dir'], i['mouse_name'], i['date'])
    offset = 1
    nframes = []
    for j in i['expts']:
        in_dir = os.path.join(i['in_dir'], str(j))
        
        xml_path = os.path.join(in_dir, "Experiment.xml")
        logging.info(f'Parsing Experiment.xml in {xml_path}')
        if os.path.exists(xml_path):
            (x, y, fr, num_frames, num_planes) = parse_xml(xml_path)
            nframes.append(num_frames)
        else:
            raise ValueError(f"Experiment.xml does not exist at {xml_path}")
            
    if run_mode == 'suite2p' or run_mode == 'all':
        from run_suite2p import run_suite2p
        ops = run_suite2p(i, results_dir, fr/num_planes, num_planes)
    if run_mode == 'deconv' or run_mode == 'all':
        from deconv import do_deconv
        do_deconv(results_dir, num_planes/fr, nframes, i['expts'], abf_dir, nplanes=num_planes)

2021-02-23 18:46:23        33938 [<ipython-input-4-364e49129f17>:       <module>():24] [21080] Parsing Experiment.xml in //mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/183628\2019-06-11\1_001\Experiment.xml


Running expt {'mouse_name': '183628', 'date': '2019-06-11', 'expts': ['1_001'], 'raw_dir': '//mohajerani-nas.uleth.ca/storage/homes/samsoon.inayat/Data/', 'tif_dir': 'E:/Userssamsoon.inayat/S_Drive/Tif_Data/', 'results_dir': 'E:/Users/samsoon.inayat/S_Drive/Processed_Data/', 'ops': {'do_bidiphase': True, 'bidiphase': 1, 'bidi_corrected': True, 'sparse_mode': False, 'diameter': 15}}
Temp directory: E:/Userssamsoon.inayat/S_Drive/Tif_Data/183628\2019-06-11\1_001
Results directory: E:/Users/samsoon.inayat/S_Drive/Processed_Data/183628\2019-06-11\1_001


2021-02-23 18:46:26        36962 [byteflow.py:       __init__():74] [21080] bytecode dump:
>          0	NOP(arg=None, lineno=19)
           2	LOAD_FAST(arg=0, lineno=19)
           4	LOAD_GLOBAL(arg=0, lineno=19)
           6	LOAD_METHOD(arg=1, lineno=19)
           8	LOAD_CONST(arg=1, lineno=19)
          10	CALL_METHOD(arg=1, lineno=19)
          12	LOAD_GLOBAL(arg=0, lineno=19)
          14	LOAD_METHOD(arg=2, lineno=19)
          16	LOAD_FAST(arg=0, lineno=19)
          18	CALL_METHOD(arg=1, lineno=19)
          20	BINARY_ADD(arg=None, lineno=19)
          22	BINARY_TRUE_DIVIDE(arg=None, lineno=19)
          24	LOAD_FAST(arg=1, lineno=19)
          26	BINARY_MULTIPLY(arg=None, lineno=19)
          28	RETURN_VALUE(arg=None, lineno=19)
2021-02-23 18:46:26        36964 [byteflow.py:            run():100] [21080] pending: deque([State(pc_initial=0 nstack_initial=0)])
2021-02-23 18:46:26        36965 [byteflow.py:            run():103] [21080] stack: []
2021-02-23 18:46:26        36965 [

{'suite2p_version': '0.9.3', 'look_one_level_down': True, 'fast_disk': [], 'delete_bin': False, 'mesoscan': False, 'bruker': False, 'h5py': [], 'h5py_key': 'data', 'save_path0': 'E:/Users/samsoon.inayat/S_Drive/Processed_Data/183628\\2019-06-11\\1_001', 'save_folder': [], 'subfolders': [], 'move_bin': False, 'nplanes': 1, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.0, 'fs': 29.16, 'force_sktiff': False, 'frames_include': -1, 'multiplane_parallel': False, 'preclassify': 0.0, 'save_mat': True, 'save_NWB': False, 'combined': True, 'aspect': 1.0, 'do_bidiphase': True, 'bidiphase': 1, 'bidi_corrected': True, 'do_registration': 1, 'two_step_registration': False, 'keep_movie_raw': False, 'nimg_init': 300, 'batch_size': 500, 'maxregshift': 0.1, 'align_by_chan': 1, 'reg_tif': False, 'reg_tif_chan2': False, 'subpixel': 10, 'smooth_sigma_time': 0, 'smooth_sigma': 1.15, 'th_badframes': 1.0, 'norm_frames': True, 'force_refImg': False, 'pad_fft': False, 'nonrigid': True, 'block_size': [128, 128],

Exception: no tiffs